In [1]:
from numpy import array
from numpy import hstack
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import callbacks
from sklearn.model_selection import train_test_split
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from IPython.display import clear_output
import datetime
import statistics
import time 
import os
import json
import yfinance as yf
from keras.models import model_from_json
import requests
from keras.models import load_model
from matplotlib import pyplot as plt

In [2]:
#takes data from yfinance
#reshapes into form (n_seq, 1, n_steps, n_features)
def data_setup(symbol,data_len,seq_len):
    end = datetime.datetime.today().strftime('%Y-%m-%d')
    start = datetime.datetime.strptime(end, '%Y-%m-%d') - datetime.timedelta(days=(data_len/0.463))
    orig_dataset = yf.download(symbol,start,end)
    close = orig_dataset['Close'].values
    open_ = orig_dataset['Open'].values
    high = orig_dataset['High'].values
    low = orig_dataset['Low'].values
    dataset,minmax = normalize_data(orig_dataset)
    cols = dataset.columns.tolist()
    data_seq = list()
    for i in range(len(cols)):
        if cols[i] < 4:
            data_seq.append(dataset[cols[i]].values)
            data_seq[i] = data_seq[i].reshape((len(data_seq[i]), 1))
    data = hstack(data_seq)
    n_steps = seq_len
    X, y = split_sequences(data, n_steps)
    n_features = X.shape[2]
    n_seq = len(X)
    n_steps = seq_len
    print(X.shape)
    X = X.reshape((n_seq,1, n_steps, n_features))
    true_y = []
    for i in range(len(y)):
        true_y.append([y[i][0],y[i][1]])
    return X,array(true_y),n_features,minmax,n_steps,close,open_,high,low, data

In [6]:
X,y,n_features,minmax,n_steps,close,open_,high,low,data = data_setup('AAPL',1500,50)

[*********************100%***********************]  1 of 1 completed
(2183, 50, 4)


In [12]:
#form (n_seq, 1, n_steps, n_features)
X.shape

(2184, 1, 50, 4)

In [4]:
def split_sequences(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            end_ix = i + n_steps
            if end_ix > len(sequences)-1:
                break
            seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)

In [ ]:
#Xarr, yarr = split_sequences(,50)

In [5]:
#normalized to max/min of each sequence instead of to the first element in that sequence 
def normalize_data(dataset):
        cols = dataset.columns.tolist()
        col_name = [0]*len(cols)
        for i in range(len(cols)):
            col_name[i] = i
        dataset.columns = col_name
        dtypes = dataset.dtypes.tolist()
#         orig_answers = dataset[attr_row_predict].values
        minmax = list()
        for column in dataset:
            dataset = dataset.astype({column: 'float32'})
        for i in range(len(cols)):
            col_values = dataset[col_name[i]]
            value_min = min(col_values)
            value_max = max(col_values)
            minmax.append([value_min, value_max])
        for column in dataset:
            values = dataset[column].values
            for i in range(len(values)):
                values[i] = (values[i] - minmax[column][0]) / (minmax[column][1] - minmax[column][0])
            dataset[column] = values
        dataset[column] = values
        return dataset,minmax

In [7]:
def enviroment_setup(X,y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        return X_train, X_test, y_train, y_test

In [8]:
X_train, X_test, y_train, y_test = enviroment_setup(X,y)

In [9]:
def initialize_network(n_steps,n_features,optimizer):
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=8, activation='relu'), input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(2))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [10]:
ConvMod=initialize_network(50,4,'adam')

In [11]:
def train_model(X_train,y_train,model,epochs):
    dirx = './ConvModel'
    os.chdir(dirx)
    h5='Stocks'+'_best_model'+'.h5'
    checkpoint = callbacks.ModelCheckpoint(h5, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
    earlystop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=epochs * 1/4, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    callback = [earlystop,checkpoint] 
    json = 'Stocks'+'_best_model'+'.json'
    model_json = model.to_json()
    with open(json, "w") as json_file:
        json_file.write(model_json)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=len(X_train)//4, verbose=2,validation_split = 0.3, callbacks = callback)
    return history

In [12]:
hist=train_model(X_train,y_train,ConvMod,300)

Epoch 1/300
3/3 - 0s - loss: 0.0531 - val_loss: 0.0098
Epoch 2/300
3/3 - 0s - loss: 0.0076 - val_loss: 0.0170
Epoch 3/300
3/3 - 0s - loss: 0.0130 - val_loss: 0.0017
Epoch 4/300
3/3 - 0s - loss: 0.0028 - val_loss: 0.0051
Epoch 5/300
3/3 - 0s - loss: 0.0066 - val_loss: 0.0053
Epoch 6/300
3/3 - 0s - loss: 0.0049 - val_loss: 0.0017
Epoch 7/300
3/3 - 0s - loss: 0.0017 - val_loss: 0.0019
Epoch 8/300
3/3 - 0s - loss: 0.0027 - val_loss: 0.0026
Epoch 9/300
3/3 - 0s - loss: 0.0022 - val_loss: 0.0010
Epoch 10/300
3/3 - 0s - loss: 0.0015 - val_loss: 0.0017
Epoch 11/300
3/3 - 0s - loss: 0.0019 - val_loss: 0.0012
Epoch 12/300
3/3 - 0s - loss: 0.0014 - val_loss: 0.0010
Epoch 13/300
3/3 - 0s - loss: 0.0013 - val_loss: 0.0011
Epoch 14/300
3/3 - 0s - loss: 0.0012 - val_loss: 9.8210e-04
Epoch 15/300
3/3 - 0s - loss: 0.0012 - val_loss: 9.4352e-04
Epoch 16/300
3/3 - 0s - loss: 0.0011 - val_loss: 9.1072e-04
Epoch 17/300
3/3 - 0s - loss: 0.0011 - val_loss: 8.8236e-04
Epoch 18/300
3/3 - 0s - loss: 0.0010 - va

In [58]:
def load_keras_model(dataset,model,loss,optimizer):
    dirx = './ConvModel'
    os.chdir(dirx)
    json_file = open('Stocks'+'_best_model'+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.compile(optimizer=optimizer, loss=loss, metrics = None)
    model.load_weights('Stocks'+'_best_model'+'.h5')
    return model
def evaluation(exe_time,X_test, y_test,X_train, y_train,history,model,optimizer,loss):
    model = load_keras_model('./ConvModel',model,loss, optimizer)
    test_loss = model.evaluate(X_test, y_test, verbose=0)
    train_loss = model.evaluate(X_train, y_train, verbose=0)
    eval_test_loss = round(100-(test_loss*100),1)
    eval_train_loss = round(100-(train_loss*100),1)
    eval_average_loss = round((eval_test_loss + eval_train_loss)/2,1)
    print("--- Training Report ---")
    #plot_loss(history)
    print('Execution time: ',round(exe_time,2),'s')
    print('Testing Accuracy:',eval_test_loss,'%')
    print('Training Accuracy:',eval_train_loss,'%')
    print('Average Network Accuracy:',eval_average_loss,'%')
    return model,eval_test_loss

In [59]:
os.getcwd()

'c:\\Users\\lesap\\Documents\\GitHub\\ConvLSTMTradingBot\\ConvModel'

In [60]:
os.chdir('..')

In [61]:
evaled_model,eval_test_loss = evaluation(5, X_test, y_test,X_train, y_train,history=hist,model=ConvMod,optimizer='adam',loss='mse')

--- Training Report ---
Execution time:  5 s
Testing Accuracy: 100.0 %
Training Accuracy: 100.0 %
Average Network Accuracy: 100.0 %


In [63]:
os.chdir('..')

In [33]:
os.getcwd()

'c:\\Users\\lesap\\Documents\\GitHub\\ConvolutionalTradingBotTensorTrade'

In [64]:
data_len=1500
end = datetime.datetime.today().strftime('%Y-%m-%d')
start = datetime.datetime.strptime(end, '%Y-%m-%d') - datetime.timedelta(days=(data_len/0.463))
symbol='AAPL'
orig_dataset = yf.download(symbol,start,end)
close = orig_dataset['Close'].values
open_ = orig_dataset['Open'].values
high = orig_dataset['High'].values
low = orig_dataset['Low'].values

bestMod=load_keras_model(orig_dataset,ConvMod,'mse','adam')

[*********************100%***********************]  1 of 1 completed


In [86]:
def market_predict(model,minmax,seq_len,n_features,n_steps,data,test_loss):
    pred_data = data[:,0].reshape((len(data[0]),1, n_steps, n_features))
    pred = model.predict(pred_data)[0]
    appro_loss = list()
    for i in range(len(pred)):
        pred[i] = pred[i] * (minmax[i][1] - minmax[i][0]) + minmax[i][0]
        appro_loss.append(((100-test_loss)/100) * (minmax[i][1] - minmax[i][0]))
    return pred,appro_loss

In [66]:
test_loss = ConvMod.evaluate(X_test, y_test, verbose=0)

In [67]:
test_loss

0.00020790912094525993

In [81]:
data.shape

(2233, 4)

In [83]:
len(data[:,2])

2233

In [87]:
pred,appro_loss = market_predict(ConvMod, minmax, 50, 4, 1500, data, test_loss)

ValueError: cannot reshape array of size 2233 into shape (4,1,1500,4)

In [21]:
import config

BASE_URL = 'https://paper-api.alpaca.markets'
API_KEY = config.api_key
SECRET_KEY = config.secret_key
ORDERS_URL = '{}/v2/orders'.format(BASE_URL)
HEADERS = {'APCA-API-KEY-ID':API_KEY,'APCA-API-SECRET-KEY':SECRET_KEY}

In [70]:
def create_order(pred_price,company,test_loss,appro_loss):
    open_price,close_price = pred_price[0],pred_price[1]
    if open_price > close_price:
        side = 'sell'
    elif open_price < close_price:
        side = 'buy'
    if side == 'buy':
        order = {
            'symbol':company,
            'qty':round(20*(test_loss/100)),
            'type':'stop_limit',
            'time_in_force':'day',
            'side': 'buy',
            'take_profit': close_price + appro_loss,
            'stop_loss': close_price - appro_loss
                }
    elif side == 'sell':
        order = {
            'symbol':company,
            'qty':round(20*(test_loss/100)),
            'type':'stop_limit',
            'time_in_force':'day',
            'side': 'sell',
            'take_profit':close_price - appro_loss,
            'stop_loss':close_price + appro_loss
                }
    r = requests.post(ORDERS_URL, json = order,headers = HEADERS)
    print(r.content)